In [1]:
import csv

from models import init_multilayer_model, build_observations, online_multilayer, big_e, Observation


In [2]:
# weights from module 7 assignment
INITIAL_WEIGHTS = [
    [  # hidden layer initial weights
        [0.3, 0.3],
        [0.3, 0.3],
    ],
    [  # output layer initial weights
        [0.8],
        [0.8],
    ],
]

INITIAL_BIASES = [
    [
        [0, 0],
    ],
    [
        [0],
    ],
]

# each observation will be a row vector of feature values and will include the
# expected output value. It is assumed that the last value of each list of values
# is the output value
INITIAL_INPUTS: list[list[float]] = [
    [1, 1, 0.9],
    [-1, -1, 0.05],
]

In [3]:
multilayer = init_multilayer_model(2, 2, INITIAL_WEIGHTS, INITIAL_BIASES, eta=1)
inputs = build_observations(INITIAL_INPUTS)

online_multilayer(multilayer, inputs, 15)
# should agree with outputs for module 7 assignment
for i, ob in enumerate(inputs):
    print(f"observation {i+1}")
    print(f"activation value: {round(multilayer(ob.features),4)}")
    print(f"Big E: {round(big_e(multilayer, ob),4)}")

observation 1
activation value: 0.6583
Big E: 0.0292
observation 2
activation value: 0.3818
Big E: 0.055


In [4]:
#random weights
multilayer = init_multilayer_model(2, 2)
inputs = build_observations(INITIAL_INPUTS)

online_multilayer(multilayer, inputs, 15000) # very large number of cycles to see convergence
for i, ob in enumerate(inputs):
    print(f"observation {i+1}")
    print(f"activation value: {round(multilayer(ob.features),4)}")
    print(f"observation value: {ob.values[0][0]}")
    print(f"Big E: {round(big_e(multilayer, ob),4)}")

observation 1
activation value: 0.8994
observation value: 0.9
Big E: 0.0
observation 2
activation value: 0.053
observation value: 0.05
Big E: 0.0


In [5]:
def read_data(filename: str) -> list[Observation]:
    with open(filename, "r") as csvfile:
        reader = csv.DictReader(csvfile)
        return build_observations([
            [float(row["Lac"]),float(row["SOW"]),float(row["TACA"])] for row in reader
        ])
data = read_data("data.csv")

In [6]:
def test_train_split(data: list[Observation]) -> tuple[list, list]:
    return data[0::2], data[1::2]

test, train = test_train_split(data)

In [7]:
test

[Observation(features=array([[0.9 , 0.87]]), values=array([[1.]])),
 Observation(features=array([[1.31, 0.75]]), values=array([[1.]])),
 Observation(features=array([[2.48, 1.14]]), values=array([[0.]])),
 Observation(features=array([[0.41, 1.87]]), values=array([[0.]])),
 Observation(features=array([[2.45, 0.52]]), values=array([[0.]])),
 Observation(features=array([[2.54, 2.97]]), values=array([[1.]])),
 Observation(features=array([[0.07, 0.09]]), values=array([[1.]])),
 Observation(features=array([[1.32, 1.96]]), values=array([[0.]])),
 Observation(features=array([[0.94, 0.34]]), values=array([[1.]])),
 Observation(features=array([[1.75, 2.21]]), values=array([[0.]]))]

In [9]:
#random weights
multilayer = init_multilayer_model(2, 2)
inputs = test

online_multilayer(multilayer, inputs, 150) # large-ish number 
print("versus test")
for i, ob in enumerate(inputs):
    print(f"observation {i+1}")
    print(f"activation value: {round(multilayer(ob.features),4)}")
    print(f"observation value: {ob.values[0][0]}")
    print(f"Big E: {round(big_e(multilayer, ob),4)}")

print()
print("versus train")
for i, ob in enumerate(train):
    print(f"observation {i+1}")
    print(f"activation value: {round(multilayer(ob.features),4)}")
    print(f"observation value: {ob.values[0][0]}")
    print(f"Big E: {round(big_e(multilayer, ob),4)}")

versus test
observation 1
activation value: 0.4855
observation value: 1.0
Big E: 0.1324
observation 2
activation value: 0.4838
observation value: 1.0
Big E: 0.1332
observation 3
activation value: 0.4735
observation value: 0.0
Big E: 0.1121
observation 4
activation value: 0.4781
observation value: 0.0
Big E: 0.1143
observation 5
activation value: 0.4786
observation value: 0.0
Big E: 0.1145
observation 6
activation value: 0.466
observation value: 1.0
Big E: 0.1426
observation 7
activation value: 0.5059
observation value: 1.0
Big E: 0.122
observation 8
activation value: 0.473
observation value: 0.0
Big E: 0.1118
observation 9
activation value: 0.4924
observation value: 1.0
Big E: 0.1288
observation 10
activation value: 0.4701
observation value: 0.0
Big E: 0.1105

versus train
observation 1
activation value: 0.4777
observation value: 0.0
Big E: 0.1141
observation 2
activation value: 0.4712
observation value: 0.0
Big E: 0.111
observation 3
activation value: 0.4695
observation value: 1.0
Big